---
# **8. Анализ результатов симуляции**

### Статистика финального капитала

Из $n_{\text{paths}}$ траекторий получаются $n_{\text{paths}}$ значений финального капитала $X_T^{(k)}$.

$$\bar{X}_T = \text{Среднее}, \quad \tilde{X}_T = \text{Медиана}, \quad \sigma_X = \text{Стандартное отклонение}$$

### Value at Risk (VaR)

VaR на уровне доверия $1-\alpha$ (обычно 95% или 99%):

$$\text{VaR}_\alpha = X_0 - q_\alpha(X_T)$$

где $q_\alpha$ — $\alpha$-квантиль (например, 5-й процентиль для 95%).

С вероятностью $1-\alpha$, потери не превысят VaR.

### Conditional Value at Risk (CVaR)

Среднее значение потерь в хвосте:

$$\text{CVaR}_\alpha = X_0 - \mathbb{E}[\text{Потери | Потери} \geq \text{VaR}_\alpha]$$

Из симуляции:
$$\text{CVaR} = \text{Среднее капитала среди худших } \alpha\% \text{ траекторий}$$

CVaR консервативнее VaR (учитывает худшие сценарии).

### Годовая доходность и волатильность

$$R^{(k)} = \left(\frac{X_T^{(k)}}{X_0}\right)^{1} - 1$$

$$\bar{R} = \text{Среднее}, \quad \sigma_R = \text{Волатильность}$$

### Sharpe Ratio

$$\text{Sharpe} = \frac{\bar{R} - r_f}{\sigma_R}$$

- Sharpe > 1 → хорошая стратегия
- Sharpe > 0 → стратегия лучше risk-free
- Sharpe < 0 → хуже, чем облигации

### Maximum Drawdown

Максимальное падение от пика:

$$\text{MDD}^{(k)} = \max_{t_1 < t_2} \frac{X_{t_1}^{(k)} - X_{t_2}^{(k)}}{X_{t_1}^{(k)}}$$

По всем путям: средний и худший MDD.

### Вероятность потерь

$$P(\text{потеря} > 10\%) = \frac{\text{Кол-во путей с } X_T < 0.9X_0}{n_{\text{paths}}}$$

---


In [ ]:
def analyze_portfolio_results(X_paths, weights_history, t_grid, asset_names, X_0,
                              rf, var_level_95=0.05, var_level_99=0.01):

    if len(X_paths) == 0:
        raise ValueError("X_paths пуст")

    if X_0 <= 0:
        raise ValueError(f"X_0 должен быть > 0, получено {X_0}")

    if rf < 0 or rf > 0.5:
        print(f"  ВНИМАНИЕ: Значение rf = {rf:.4f} выглядит необычно")
        print(f"   rf должна быть в долях (например, 0.1394 для 13.94%)")
        print(f"   Убедитесь, что она совпадает с rf_hjb из блока 3.1\n")

    final_capitals = X_paths[:, -1]
    T_years = t_grid[-1] - t_grid[0]
    n_paths = len(X_paths)

    mean_final = np.mean(final_capitals)
    median_final = np.median(final_capitals)
    std_final = np.std(final_capitals, ddof=1)
    min_final = np.min(final_capitals)
    max_final = np.max(final_capitals)

    confidence_level_95 = int((1 - var_level_95) * 100)
    var_capital_95 = np.quantile(final_capitals, var_level_95)
    loss_var_95 = X_0 - var_capital_95
    loss_var_percent_95 = (loss_var_95 / X_0) * 100

    tail_capitals_95 = final_capitals[final_capitals <= var_capital_95]


    if len(tail_capitals_95) > 0:
        tail_losses_95 = X_0 - tail_capitals_95
        loss_cvar_95 = np.mean(tail_losses_95)
        cvar_capital_95 = X_0 - loss_cvar_95
        loss_cvar_percent_95 = (loss_cvar_95 / X_0) * 100
    else:
        loss_cvar_95 = 0
        cvar_capital_95 = X_0
        loss_cvar_percent_95 = 0

    confidence_level_99 = int((1 - var_level_99) * 100)
    var_capital_99 = np.quantile(final_capitals, var_level_99)
    loss_var_99 = X_0 - var_capital_99
    loss_var_percent_99 = (loss_var_99 / X_0) * 100

    tail_capitals_99 = final_capitals[final_capitals <= var_capital_99]


    if len(tail_capitals_99) > 0:
        tail_losses_99 = X_0 - tail_capitals_99
        loss_cvar_99 = np.mean(tail_losses_99)
        cvar_capital_99 = X_0 - loss_cvar_99
        loss_cvar_percent_99 = (loss_cvar_99 / X_0) * 100
    else:
        loss_cvar_99 = 0
        cvar_capital_99 = X_0
        loss_cvar_percent_99 = 0

    def compute_max_drawdown(path):
        running_max = np.maximum.accumulate(path)
        valid_mask = running_max > 1e-6

        if not np.any(valid_mask):
            return 1.0

        drawdown = np.zeros_like(path)
        drawdown[valid_mask] = (running_max[valid_mask] - path[valid_mask]) / running_max[valid_mask]

        return np.max(drawdown)

    max_drawdowns = np.array([compute_max_drawdown(path) for path in X_paths])
    mean_max_drawdown = np.mean(max_drawdowns)
    worst_drawdown = np.max(max_drawdowns)

    returns = np.zeros(len(final_capitals))

    if T_years <= 0:
        raise ValueError(f"T_years должен быть > 0, получено {T_years}")


    for i, cap in enumerate(final_capitals):
        if cap > 1e-10:
            returns[i] = (cap / X_0) ** (1 / T_years) - 1
        else:
            returns[i] = -1.0

    mean_return = np.mean(returns)
    median_return = np.median(returns)
    std_return = np.std(returns, ddof=1)

    if std_return > 1e-10:
        sharpe = (mean_return - rf) / std_return
    else:
        sharpe = np.nan
        print(f"  Волатильность близка к нулю, Sharpe Ratio не определен")

    prob_loss_10 = np.sum(final_capitals < 0.9 * X_0) / n_paths
    prob_loss_50 = np.sum(final_capitals < 0.5 * X_0) / n_paths

    mean_weights_final = np.mean(weights_history[:, -1, :], axis=0)
    std_weights_final = np.std(weights_history[:, -1, :], axis=0, ddof=1)
    median_weights_final = np.median(weights_history[:, -1, :], axis=0)

    print(f"\n ФИНАЛЬНЫЙ КАПИТАЛ (T = {T_years:.2f} лет):\n")
    print(f"   Начальный:            {X_0:>15,.0f} ₽")
    print(f"   Среднее:              {mean_final:>15,.0f} ₽  ({(mean_final/X_0 - 1)*100:>7.2f}%)")
    print(f"   Медиана:              {median_final:>15,.0f} ₽  ({(median_final/X_0 - 1)*100:>7.2f}%)")
    print(f"   Минимум:              {min_final:>15,.0f} ₽  ({(min_final/X_0 - 1)*100:>7.2f}%)")
    print(f"   Максимум:             {max_final:>15,.0f} ₽  ({(max_final/X_0 - 1)*100:>7.2f}%)")

    print(f"\n МЕТРИКИ РИСКА:\n")

    print(f"   VaR ({confidence_level_95}% confidence level):")
    print(f"     • Капитал:          {var_capital_95:>15,.0f} ₽  (5-й процентиль)")
    print(f"     • Потери (абс):     {loss_var_95:>15,.0f} ₽")
    print(f"     • Потери (%):       {loss_var_percent_95:>15.2f}%")
    print(f"\n   CVaR ({confidence_level_95}% confidence level):")
    print(f"     • Среднее потери:   {loss_cvar_95:>15,.0f} ₽  (средние в хвосте)")
    print(f"     • Потери (%):       {loss_cvar_percent_95:>15.2f}%")
    print(f"     • Эквив. капитал:   {cvar_capital_95:>15,.0f} ₽")

    print(f"\n   VaR ({confidence_level_99}% confidence level):")
    print(f"     • Капитал:          {var_capital_99:>15,.0f} ₽  (1-й процентиль)")
    print(f"     • Потери (абс):     {loss_var_99:>15,.0f} ₽")
    print(f"     • Потери (%):       {loss_var_percent_99:>15.2f}%")
    print(f"\n   CVaR ({confidence_level_99}% confidence level):")
    print(f"     • Среднее потери:   {loss_cvar_99:>15,.0f} ₽  (средние в хвосте)")
    print(f"     • Потери (%):       {loss_cvar_percent_99:>15.2f}%")
    print(f"     • Эквив. капитал:   {cvar_capital_99:>15,.0f} ₽")

    print(f"\n   Maximum Drawdown:")
    print(f"     • Средний:          {mean_max_drawdown:>15.2%}")
    print(f"     • Худший:           {worst_drawdown:>15.2%}")

    print(f"\n МЕТРИКИ ДОХОДНОСТИ (r_f = {rf*100:.2f}%, из блока 3.1):\n")
    print(f"   Средняя годовая:       {mean_return:>15.2%}")
    print(f"   Медианная годовая:     {median_return:>15.2%}")
    print(f"   Волатильность:         {std_return:>15.2%}")

    if not np.isnan(sharpe):
        status = "✅" if sharpe > 0 else "⚠️"
        print(f"   {status} Sharpe Ratio (r_f учтена): {sharpe:>15.4f}")
        if sharpe < 0:
            print(f"      (отрицательный — средняя доходность ниже r_f)")
    else:
        print(f"   Sharpe Ratio:          {sharpe:>15}  (не определен)")

    print(f"   P(потеря > 10%):       {prob_loss_10:>15.2%}")
    print(f"   P(потеря > 50%):       {prob_loss_50:>15.2%}")

    print(f"\n ОПТИМАЛЬНЫЕ ВЕСА (в конце периода, пассивная эволюция):\n")
    for name, w_mean, w_median, w_std in zip(asset_names, mean_weights_final,
                                              median_weights_final, std_weights_final):
        emoji = "🟢" if w_mean > 0.01 else "⚪"
        print(f"   {emoji} {name:5s}: μ={w_mean*100:>6.2f}% | median={w_median*100:>6.2f}% | σ={w_std*100:>5.2f}%")

    return {
        'mean_final': mean_final,
        'median_final': median_final,
        'min_final': min_final,
        'max_final': max_final,
        'std_final': std_final,

        'var_capital_95': var_capital_95,
        'loss_var_95': loss_var_95,
        'loss_var_percent_95': loss_var_percent_95,
        'cvar_capital_95': cvar_capital_95,
        'loss_cvar_95': loss_cvar_95,
        'loss_cvar_percent_95': loss_cvar_percent_95,

        'var_capital_99': var_capital_99,
        'loss_var_99': loss_var_99,
        'loss_var_percent_99': loss_var_percent_99,
        'cvar_capital_99': cvar_capital_99,
        'loss_cvar_99': loss_cvar_99,
        'loss_cvar_percent_99': loss_cvar_percent_99,

        'mean_max_drawdown': mean_max_drawdown,
        'worst_drawdown': worst_drawdown,

        'mean_return': mean_return,
        'median_return': median_return,
        'std_return': std_return,
        'sharpe': sharpe,

        'prob_loss_10': prob_loss_10,
        'prob_loss_50': prob_loss_50,

        'optimal_weights': mean_weights_final,
        'weights_median': median_weights_final,
        'weights_std': std_weights_final,

        'rf': rf,
        'T_years': T_years,
        'n_paths': n_paths
    }

# **Визуализация**

In [ ]:
def visualize_results(X_paths, weights_history, t_grid, asset_names, X_0, var_level=0.05):

    if len(X_paths) == 0:
        raise ValueError("X_paths пуст — нет данных для визуализации")

    if X_0 <= 0:
        raise ValueError(f"Начальный капитал должен быть положительным (X_0={X_0})")

    final_capitals = X_paths[:, -1]
    var_threshold = np.quantile(final_capitals, var_level)
    median_final = np.median(final_capitals)
    mean_final = np.mean(final_capitals)
    mean_path = np.mean(X_paths, axis=0)
    std_path = np.std(X_paths, axis=0)

    q05_path = np.quantile(X_paths, 0.05, axis=0)
    q25_path = np.quantile(X_paths, 0.25, axis=0)
    q75_path = np.quantile(X_paths, 0.75, axis=0)
    q95_path = np.quantile(X_paths, 0.95, axis=0)

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.hist(final_capitals, bins=100, color='steelblue', alpha=0.7, edgecolor='black', linewidth=0.5)

    ax.axvline(X_0, color='green', linestyle=':', linewidth=2.5, label=f'Начальный капитал ({X_0:,.0f} ₽)')
    ax.axvline(median_final, color='red', linestyle='--', linewidth=2, label=f'Медиана ({median_final:,.0f} ₽)')
    ax.axvline(mean_final, color='orange', linestyle='--', linewidth=2, label=f'Среднее ({mean_final:,.0f} ₽)')
    ax.axvline(var_threshold, color='purple', linestyle='--', linewidth=2,
               label=f'VaR {int((1-var_level)*100)}% ({var_threshold:,.0f} ₽)')

    ax.set_xlabel('Финальный капитал (₽)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Число симуляций', fontsize=12, fontweight='bold')
    ax.set_title('Распределение финальных капиталов портфеля', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='best')
    ax.grid(alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(12, 6))

    ax.fill_between(t_grid, q05_path, q95_path, alpha=0.2, color='lightblue', label='90% интервал (5%-95%)')
    ax.fill_between(t_grid, q25_path, q75_path, alpha=0.3, color='steelblue', label='50% интервал (25%-75%)')

    median_path = np.median(X_paths, axis=0)
    ax.plot(t_grid, mean_path, color='darkblue', linewidth=2.5, label='Среднее', zorder=5)
    ax.plot(t_grid, median_path, color='red', linewidth=2, linestyle='--', label='Медиана', zorder=5)

    ax.axhline(X_0, color='green', linestyle=':', linewidth=2, label=f'Начальный капитал ({X_0:,.0f} ₽)', zorder=3)

    ax.set_xlabel('Время (годы)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Капитал (₽)', fontsize=12, fontweight='bold')
    ax.set_title('Эволюция капитала портфеля во времени', fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='best')
    ax.grid(alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(12, 6))

    perc_95 = np.percentile(final_capitals, 95)
    filtered_indices = final_capitals <= perc_95
    filtered_paths = X_paths[filtered_indices]

    n_paths_to_plot = min(100, filtered_paths.shape[0])
    sample_indices = np.random.choice(filtered_paths.shape[0], n_paths_to_plot, replace=False)

    for idx in sample_indices:
        ax.plot(t_grid, filtered_paths[idx], alpha=0.3, linewidth=0.7, color='gray')

    ax.plot(t_grid, mean_path, color='darkblue', linewidth=3, label='Среднее', zorder=10)
    ax.axhline(X_0, color='green', linestyle=':', linewidth=2, label=f'Начальный капитал', zorder=5)

    ax.set_xlabel('Время (годы)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Капитал (₽)', fontsize=12, fontweight='bold')
    ax.set_title(f'Примеры траекторий портфеля (до 95% перцентиля, n={n_paths_to_plot})',
                 fontsize=14, fontweight='bold')
    ax.legend(fontsize=10, loc='best')
    ax.grid(alpha=0.3, linestyle='--')
    plt.tight_layout()
    plt.show()

    fig, ax = plt.subplots(figsize=(12, 6))

    mean_weights_over_time = np.mean(weights_history, axis=0)

    ax.stackplot(t_grid, mean_weights_over_time.T, labels=asset_names, alpha=0.8)

    ax.set_xlabel('Время (годы)', fontsize=12, fontweight='bold')
    ax.set_ylabel('Вес в портфеле', fontsize=12, fontweight='bold')
    ax.set_title('Эволюция весов активов во времени (средние по траекториям)', fontsize=14, fontweight='bold')
    ax.set_ylim(0, 1)
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=10)
    ax.grid(alpha=0.3, linestyle='--', axis='y')
    plt.tight_layout()
    plt.show()

# **Главный запуск pipeline**

In [ ]:
required_objects = {
    'HJBStrategy': 'HJBStrategy',
    'simulate_portfolio_passive': 'simulate_portfolio_passive',
    'analyze_portfolio_results': 'analyze_portfolio_results',
    'visualize_results': 'visualize_results'
}

missing_objects = []
for name, var_name in required_objects.items():
    try:
        obj = eval(var_name)
        if not callable(obj) and not isinstance(obj, type):
            missing_objects.append(name)
    except NameError:
        missing_objects.append(name)

if missing_objects:
    error_msg = (
        f" Следующие необходимые объекты не определены:\n"
        f"   {', '.join(missing_objects)}\n\n"
        f"Убедитесь, что выполнены предыдущие блоки:\n"
        f"   • Блок 3.1: HJBStrategy класс + rf_hjb\n"
        f"   • Блок 3.2: simulate_portfolio_passive функция\n"
        f"   • Блок 3.3: analyze_portfolio_results функция\n"
        f"   • Блок 3.4: visualize_results функция\n"
    )
    raise NameError(error_msg)

try:
    rf_from_block_31 = rf_hjb
    print(f" Обнаружена безрисковая ставка из блока 3.1:")
    print(f"   rf_hjb = {rf_from_block_31:.4f} ({rf_from_block_31*100:.2f}%)\n")
except NameError:
    error_msg = (
        f" Переменная rf_hjb не определена (должна быть из блока 3.1)\n\n"
        f"Убедитесь, что выполнен блок 3.1 полностью.\n"
        f"Блок 3.1 должен определить rf_hjb как:\n"
        f"   rf_manual = 13.94\n"
        f"   rf_hjb = rf_manual / 100\n"
    )
    raise NameError(error_msg)

try:
    mu = mu_with_dividends.values if hasattr(mu_with_dividends, 'values') else mu_with_dividends
    cov_matrix_array = filtered_cov.values if hasattr(filtered_cov, 'values') else filtered_cov
    asset_names = list(filtered_names)
except NameError as e:
    error_msg = (
        f" Переменная не определена: {e}\n\n"
        f"Убедитесь, что выполнены предыдущие блоки:\n"
        f"   • Блок 2.1.2: расчёт cov_matrix\n"
        f"   • Блок 2.3: фильтрация (filtered_cov, filtered_names)\n"
        f"   • Блок 2.4: дивиденды (mu_with_dividends)\n"
    )
    raise NameError(error_msg)

mu = np.asarray(mu).flatten()
cov_matrix_array = np.asarray(cov_matrix_array)
n_assets = len(asset_names)

if mu.shape[0] != n_assets:
    raise ValueError(
        f"Несоответствие размерностей: μ содержит {mu.shape[0]} элементов, "
        f"но активов {n_assets}"
    )

if cov_matrix_array.shape != (n_assets, n_assets):
    raise ValueError(
        f"Некорректная размерность ковариационной матрицы: "
        f"{cov_matrix_array.shape}, ожидается ({n_assets}, {n_assets})"
    )

print(f" Входные данные валидированы:\n")
print(f"   Активов:         {n_assets}")
print(f"   Размерность μ:   {mu.shape}")
print(f"   Размерность Σ:   {cov_matrix_array.shape}")
print(f"   r_f (из блока 3.1): {rf_from_block_31*100:.2f}%\n")

print(f" Ожидаемые годовые доходности (μ = capital gains + dividends):\n")

negative_mu_count = np.sum(mu < 0)
all_negative = np.all(mu < 0)

for i, ticker in enumerate(asset_names):
    emoji = "🟢" if mu[i] > 0 else "🔴"
    print(f"   {emoji} {ticker:5s}: {mu[i]*100:>7.2f}%")

print(f"\n   Средняя μ: {np.mean(mu)*100:>7.2f}%")
print(f"   Мин. μ:    {np.min(mu)*100:>7.2f}% ({asset_names[np.argmin(mu)]})")
print(f"   Макс. μ:   {np.max(mu)*100:>7.2f}% ({asset_names[np.argmax(mu)]})")

if all_negative:
    print(f"\n  КРИТИЧЕСКОЕ ВНИМАНИЕ: Все ожидаемые доходности отрицательны!")
    print(f"   Портфель будет работать убыточно.")
    print(f"   Проверьте входные данные (capital gains + dividends)")
elif negative_mu_count > n_assets / 2:
    print(f"\n  ВНИМАНИЕ: {negative_mu_count}/{n_assets} активов имеют отрицательные доходности")

excess_mu = mu - rf_from_block_31
print(f"\n Избыточные доходности (μ - r_f = μ - {rf_from_block_31*100:.2f}%):\n")

positive_excess_count = np.sum(excess_mu > 0)
for i, ticker in enumerate(asset_names):
    excess = excess_mu[i]
    emoji = "🟢" if excess > 0 else "🔴"
    print(f"   {emoji} {ticker:5s}: {excess*100:>7.2f}%")

print(f"\n   Активов с положительной избыточной доходностью: {positive_excess_count}/{n_assets}")

if positive_excess_count == 0:
    print(f"\n  ВНИМАНИЕ: Ни один актив не имеет положительную избыточную доходность (μ > r_f)")
    print(f"   Оптимальное решение может быть инвестировать только в безрисковый актив.")
    print(f"   Проверьте данные или увеличьте временной горизонт.")

X_0 = 10_593_820_754_282,10
gamma = 3.0
random_seed = 42

n_paths = 5000
T = 1.0
n_steps = 252

t_grid = np.linspace(0, T, n_steps)
dt = T / (n_steps - 1)

time_unit = 'год' if T == 1 else 'года/лет'

print(f"  Параметры портфеля:")
# print("   X₀ (начальный капитал): {:>12,.0f}".format(X_0))
print(f"   γ (риск-аверсия):        {gamma:>12.1f}")
print(f"   Горизонт (T):             {T:>12.1f} {time_unit}")

print(f"\n  Интерпретация γ = {gamma}:")
if gamma == 1.0:
    print(f"   → Логарифмическая полезность (нейтральный инвестор)")
elif gamma < 3.0:
    print(f"   → Умеренная риск-аверсия (агрессивный инвестор)")
elif gamma < 10.0:
    print(f"   → Стандартная риск-аверсия (обычный инвестор)")
else:
    print(f"   → Высокая риск-аверсия (консервативный инвестор)")

print(f"\n  Параметры симуляции:")
print(f"   Траекторий (n_paths):    {n_paths:>12,}")
print(f"   Временных шагов:         {n_steps:>12}")
print(f"   Шаг времени (dt):        {dt:>12.4f} лет ({dt*252:.0f} день)")
print(f"   Random seed:             {random_seed:>12}")

eigenvalues = np.linalg.eigvalsh(cov_matrix_array)
min_eig = eigenvalues.min()
max_eig = eigenvalues.max()
condition_number = max_eig / min_eig if min_eig > 0 else np.inf

if min_eig > 1e-10:
    L = np.linalg.cholesky(cov_matrix_array)
    print(f" Матрица положительно определена")
    print(f"   λ_min = {min_eig:.2e}")
    print(f"   λ_max = {max_eig:.2e}")
    print(f"   Condition number = {condition_number:.2e}")
    print(f"   L.shape = {L.shape}")
else:
    print(f"  ВНИМАНИЕ: Матрица близка к вырожденной")
    print(f"   λ_min = {min_eig:.2e}")
    print(f"   Condition number = {condition_number:.2e}")
    print(f"\n   Возможные причины:")
    print(f"   • Недостаточно данных (малое число наблюдений)")
    print(f"   • Коллинеарность активов (высокие корреляции)")
    print(f"   • Ошибки в расчете матрицы")

    epsilon = max(1e-6, abs(min_eig) * 10)
    cov_matrix_reg = cov_matrix_array + epsilon * np.eye(n_assets)
    L = np.linalg.cholesky(cov_matrix_reg)

    print(f"\n   Применена регуляризация:")
    print(f"   ε = {epsilon:.2e}")
    print(f"   L.shape = {L.shape}")
    print(f"\n     ВНИМАНИЕ: РЕЗУЛЬТАТЫ МОГУТ БЫТЬ НЕНАДЕЖНЫ")
    print(f"   Рекомендуется проверить входные данные")

print(f"\n Передача безрисковой ставки в стратегию:")
print(f"   rf_hjb = {rf_from_block_31:.4f} ({rf_from_block_31*100:.2f}%)")

try:
    strategy = HJBStrategy(
        mu=mu,
        cov_matrix=cov_matrix_array,
        risk_free_rate=rf_from_block_31,
        gamma=gamma,
        asset_names=asset_names
    )
    print(f"\n HJBStrategy инициализирована с rf = {rf_from_block_31*100:.2f}%")
except Exception as e:
    print(f"\n Ошибка при инициализации HJBStrategy: {e}")
    raise

print(f"\n Оптимальные веса портфеля (после инициализации):")
strategy.print_weights()

start_time = time.time()

try:
    X_paths, weights_history, t_grid = simulate_portfolio_passive(
        mu=mu,
        sigma_cholesky=L,
        X_0=X_0,
        strategy=strategy,
        asset_names=asset_names,
        t_grid=t_grid,
        n_paths=n_paths,
        random_seed=random_seed
    )
except Exception as e:
    print(f" Ошибка при симуляции: {e}")
    raise

elapsed = time.time() - start_time

print(f"   Производительность: {n_paths * n_steps / elapsed:,.0f} шагов/сек")
print(f"   X_paths:          {X_paths.shape}")
print(f"   weights_history:  {weights_history.shape}")

try:
    results = analyze_portfolio_results(
        X_paths=X_paths,
        weights_history=weights_history,
        t_grid=t_grid,
        asset_names=asset_names,
        X_0=X_0,
        rf=rf_from_block_31,
        var_level_95=0.05,
        var_level_99=0.01
    )

    if abs(results['rf'] - rf_from_block_31) > 1e-6:
        raise ValueError(
            f"Несоответствие безрисковой ставки: "
            f"блок 3.1 = {rf_from_block_31:.4f}, "
            f"результаты = {results['rf']:.4f}"
        )

except Exception as e:
    print(f" Ошибка при анализе результатов: {e}")
    raise

try:
    visualize_results(
        X_paths=X_paths,
        weights_history=weights_history,
        t_grid=t_grid,
        asset_names=asset_names,
        X_0=X_0,
        var_level=0.05
    )
except Exception as e:
    print(f"  Ошибка при визуализации (графики пропущены): {e}")
    print(f"   Программа продолжает работу без графиков\n")

print(f" Основные результаты:\n")
print(f"   Начальный капитал:         {X_0:>15,.0f} ₽")
print(f"   Средний финальный капитал: {results['mean_final']:>15,.0f} ₽")
print(f"     → Прирост:               {(results['mean_final']/X_0 - 1)*100:>+15.2f}%")
print(f"\n   Медианный финальный:       {results['median_final']:>15,.0f} ₽")
print(f"     → Прирост:               {(results['median_final']/X_0 - 1)*100:>+15.2f}%")
print(f"\n   Средняя годовая доходность: {results['mean_return']*100:>14.2f}%")
print(f"   Волатильность доходности:  {results['std_return']*100:>14.2f}%")
print(f"   Sharpe Ratio (r_f={rf_from_block_31*100:.2f}%): {results['sharpe']:>22.4f}")

print(f"\n МЕТРИКИ РИСКА (VaR и CVaR):\n")

print(f"   VaR (95% confidence - 5-й процентиль):")
print(f"     • Капитал:          {results['var_capital_95']:>15,.0f} ₽")
print(f"     • Потери:           {results['loss_var_95']:>15,.0f} ₽  ({results['loss_var_percent_95']:>6.2f}%)")
print(f"   CVaR (95% confidence):")
print(f"     • Капитал:          {results['cvar_capital_95']:>15,.0f} ₽")
print(f"     • Потери:           {results['loss_cvar_95']:>15,.0f} ₽  ({results['loss_cvar_percent_95']:>6.2f}%)")

print(f"\n   VaR (99% confidence - 1-й процентиль):")
print(f"     • Капитал:          {results['var_capital_99']:>15,.0f} ₽")
print(f"     • Потери:           {results['loss_var_99']:>15,.0f} ₽  ({results['loss_var_percent_99']:>6.2f}%)")
print(f"   CVaR (99% confidence):")
print(f"     • Капитал:          {results['cvar_capital_99']:>15,.0f} ₽")
print(f"     • Потери:           {results['loss_cvar_99']:>15,.0f} ₽  ({results['loss_cvar_percent_99']:>6.2f}%)")

print(f"\n   Maximum Drawdown:")
print(f"     • Средний:          {results['mean_max_drawdown']:>15.2%}")
print(f"     • Худший:           {results['worst_drawdown']:>15.2%}")

print(f"\n  Производительность:")
print(f"   Траекторий:                {n_paths:>15,}")
print(f"   Временных шагов:           {n_steps:>15,}")
print(f"   Время выполнения:          {elapsed:>15.2f} сек")
print(f"   Скорость:                  {n_paths * n_steps / elapsed:>15,.0f} шагов/сек")

print(f"\n Доступные переменные для дальнейшего анализа:\n")
print(f"   X_paths          : {X_paths.shape}")
print(f"      • Размер: (n_paths, num_steps) = ({n_paths}, {n_steps})")
print(f"      • Использование: X_paths[path_idx, time_idx] = капитал в момент времени")
print(f"      • Пример: final_capital = X_paths[0, -1]  # финальный капитал 1-й траектории")
print(f"\n   weights_history  : {weights_history.shape}")
print(f"      • Размер: (n_paths, num_steps, n_assets) = ({n_paths}, {n_steps}, {n_assets})")
print(f"      • Использование: weights_history[path_idx, time_idx, asset_idx] = вес актива")
print(f"      • Пример: final_weights = weights_history[0, -1, :]  # веса в конце 1-й траектории")
print(f"\n   results          : dict")
print(f"      • Метрики: mean_final, median_final, std_return, sharpe, var_capital_95/99, etc.")
print(f"      • Пример: print(results['sharpe'])  # Sharpe Ratio (с r_f = {rf_from_block_31*100:.2f}%)")
print(f"      • Новые метрики: var_capital_99, loss_var_99, cvar_capital_99, loss_cvar_99")
print(f"\n   strategy         : HJBStrategy")
print(f"      • Объект стратегии с параметрами γ = {gamma}")
print(f"      • r_f = {rf_from_block_31*100:.2f}% (из блока 3.1)")
print(f"      • Методы: get_optimal_weights(), get_portfolio_stats()")
print(f"\n   t_grid           : {t_grid.shape}")
print(f"      • Временная сетка: от {t_grid[0]:.4f} до {t_grid[-1]:.4f} лет")

print(f"\n КОНСИСТЕНТНОСТЬ МЕЖДУ БЛОКАМИ:")
print(f"    rf_hjb:           {rf_from_block_31:.4f} ({rf_from_block_31*100:.2f}%)")
print(f"    r_f (HJBStrategy):            {strategy.risk_free_rate:.4f} ({strategy.risk_free_rate*100:.2f}%)")
print(f"    r_f (результаты анализа):    {results['rf']:.4f} ({results['rf']*100:.2f}%)")
print(f"    Sharpe Ratio:                {results['sharpe']:.4f} (с учётом r_f)")
print(f"    VaR/CVaR:                    95% и 99% уровни рассчитаны")